In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import inspect
import getpass
import json

import pandas as pd

project_path = os.path.abspath( os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe()))) + "/../" )
sys.path.insert(0, project_path)
print(project_path)

from database_storage.helper import *

from database_storage.database.database import *
from database_storage.main import *

from database_storage.objects.human import *
from database_storage.objects.issue import *
from database_storage.objects.kpi import *
from database_storage.objects.machine import *
from database_storage.objects.tag import *
from database_storage.objects.maintenanceworkorder import *


/Users/sam11/Git/nestor


# Setup Dataset informations

foreach Dataset get all the specific information such as:

    user : Identification to connect to the database -STRING
    password : Password to connect to the database -STRING
    
    csv_file : Path of the raw dataset (the readable one created by Nestor) -CSV
    csvSchemas_file : Path of the file use to link the header of the CSV with the properties in the database -YAML
    
    bincsv_file : Path of the 1GramTag Binnary csv (created by Nestor) -CSV
    binNcsv_file : Path of the NGramTag Binnary csv (created by Nestor) -CSV
    
    vocabcsv_file : Path of the vocabulary 1Gram file (created by Nestor) -CSV
    vocabNcsv_file : Path of the vocabulary NGram file (created by Nestor) -CSV
    
    tree_path : Path of the item-item hierarchy (created by <insert a name here>) -JSON

### Mine

In [ ]:
user = "neo4j"
password = 'MINE!!'

csv_file = os.path.join(project_path ,'data', 'mine_data', 'readable.csv')
csvSchemas_file = os.path.join(project_path , 'data',  'mine_data', 'mine_header.yaml')


bincsv_file = os.path.join(project_path ,'data', 'mine_data', 'bin_1g.csv')
binNcsv_file = os.path.join(project_path ,'data', 'mine_data', 'bin_Ng.csv')

vocabcsv_file = os.path.join(project_path ,'data', 'mine_data', 'vocab_1g.csv')
vocabNcsv_file = os.path.join(project_path ,'data', 'mine_data', 'vocab_2g.csv')

bolt = bolt://localhost:7687"

In [ ]:
tree_path = os.path.join(project_path ,'data', 'mine_data', 'item_tree.json')

### Others

In [ ]:
#user = getpass.getuser()
password = getpass.getpass("Password:")

# Load Files from Path

foreach path file, load them into the memory using the needed format:
    
    Dataframe for : csv_file, bincsv_file, binNcsv_file, vocabcsv_file, vocabNcsv_file
   
    Dictionary for : csvSchemas_file, tree_path

In [4]:
dataframe = pd.read_csv(csv_file, index_col=0, header=0)
dataframe.fillna("", inplace=True)


dataframe_bincsv = pd.read_csv(bincsv_file, index_col=0, header=[0,1])
dataframe_binNcsv = pd.read_csv(binNcsv_file, index_col=0, header=[0,1])

dataframe_vocab1g = pd.read_csv(vocabcsv_file, index_col=0)
dataframe_vocabNg = pd.read_csv(vocabNcsv_file, index_col=0)

csvSchemas_dict = openYAMLFile(csvSchemas_file)

yaml file open


In [5]:
with open(tree_path) as file:
    tree_dict = json.load(file)

NameError: name 'tree_path' is not defined

# ConnectGraphDatabase

In [6]:
databaseSchema_file = os.path.join(project_path ,'database_storage','database', 'DatabaseSchema.yaml')
databaseSchema_dict = openYAMLFile(databaseSchema_file)
database = DatabaseNeo4J(bolt, user, password, databaseSchema_dict) 

yaml file open


## Clean the database

In [7]:
database.deleteData()

database.dropConstraints()
database.dropIndexes()

database.createIndexes()
database.createConstraints()

1

## Create Cypher Queries

### Create historical_data

In [8]:
database.runQueries(cypherCreate_historicalMaintenanceWorkOrder (database.schema, dataframe, csvSchemas_dict))

100%|██████████| 3438/3438 [01:38<00:00, 34.80it/s]


1

### Create tag_data

In [9]:
database.runQueries(cypherCreate_tag(database, dataframe_bincsv, dataframe_vocab1g, dataframe_vocabNg))

100%|██████████| 796/796 [00:16<00:00, 47.72it/s]


1

In [10]:
database.runQueries(cypherCreate_tag(database, dataframe_binNcsv, dataframe_vocab1g, dataframe_vocabNg))

100%|██████████| 1009/1009 [00:10<00:00, 92.27it/s]


1

### Update 1gram-->Ngram

In [11]:
database.runQueries(cypherLink_Ngram1gram(database))

100%|██████████| 6/6 [00:00<00:00, 13.60it/s]


1

### Update issue-->item

In [12]:
database.runQueries(cypherLink_itemIssue(database))

100%|██████████| 2/2 [00:00<00:00,  2.87it/s]


1

### Update item-->item

In [ ]:
database.runQueries(cypherCreate_itemsTree(database,tree_dict ))

------------------

In [ ]:
r = cypherCreate_historicalMaintenanceWorkOrder (database, dataframe, csvSchemas_dict)